# Background

- **Author**: `<AUTHOR NAME>`
- **Created At**: `<YYYY-MM-DD>`
- **Research Motivation and Context (why are we interested in the findings?)：**
- **Main Findings and Takeaways：**
- **Future Direciton：**

In [ ]:
# Load packages here
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt


In [ ]:
# Load input data here and please finish all the data manipulation here.
# Finish this block by printing the first ten observations of the data.
# Note:
# - You may only read data from /data/processed.
# - Files in /data/processed should already be cleaned and prepared for analysis.
# - Beyond simple filtering of observations or generating a small number of variables,
#   further data manipulation is not allowed. If more extensive changes are needed,
#   update the source data instead.

df = pd.read_csv("/data/processed/speed-mht.csv")
df.head(10)

In [ ]:
# Summary statistics block
df.describe(include='all')

### The actual analysis starts below
Make the graphs, summary statistics, regression model below.
Make sure you have followed the guidelines as specified in 
[專案資料夾結構、檔案命名與文件規範](https://docs.google.com/document/d/1sl6gEFMdmiGsiNjLe17UmZ30xKxq15U0Mb2B-Jvusxg/edit?tab=t.33iie8ybx7s4).


In [ ]:
# === Weekly weighted speed analysis ===
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# === 0. 設置初始位置 ===
os.chdir(os.path.dirname(os.path.abspath(__file__)))

# === 1. 讀取資料 ===
df = pd.read_csv("speed-mht.csv")

# === 2. 將日期轉成 datetime ===
df["DATE"] = pd.to_datetime(df["DATA_AS_OF"]).dt.date
df["DATE"] = pd.to_datetime(df["DATE"])

# === 3. 分類 CBD 與否 ===
df["CBD_group"] = df["is_in_CBD"].apply(
    lambda x: "in_CBD" if "CBD" in str(x) and "not" not in x.lower() else "not_in_CBD"
)

# === 4. 用速度與時間求距離 ===
df["DISTANCE_M"] = df["SPEED"] * df["TRAVEL_TIME"]

# === 5. 建立 WEEK 欄位 ===
df["WEEK"] = df["DATE"].dt.to_period("W").apply(lambda r: r.start_time)

# === 6. 週加權平均速度計算 ===
weekly = df.groupby(["WEEK", "CBD_group"]).apply(
    lambda g: g["DISTANCE_M"].sum() / g["TRAVEL_TIME"].sum()
).unstack()

weekly = weekly * 3.6

# === 7. 輸出到 CSV ===
weekly.to_csv("weekly_weighted_avg_speed_by_CBD.csv")

# === 8. 畫折線圖 ===
plt.figure(figsize=(12, 6))
plt.plot(weekly.index, weekly["in_CBD"], label="in_CBD")
plt.plot(weekly.index, weekly["not_in_CBD"], label="not_in_CBD")

plt.xlabel("Week")
plt.ylabel("Weighted Avg Speed (km/h)")
plt.title("Weekly Weighted Average Speed: in_CBD vs not_in_CBD")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.axvline(
    x=dt.datetime(2025, 1, 5),
    color='red',
    linestyle='--',
    linewidth=1.5,
    label='CBD policy starts (2025-01-05)'
)

plt.legend()
plt.savefig("weekly_weighted_speed_plot.png")
